# <font color='black'>Методы анализа неоднородных данных и паттерн-анализ. Практическое занятие 1 </font>
## <font color='black'>Линейная регрессия: основы </font>
В рамках данного практического занятия мы обратимся к данным из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Возьмем такой массив, так как мы пока не знакомились с моделями для анализа панельных данных.

Стоит отметить, что авторы изучают взаимосвязь уровня коррупции (является откликом в регрессионной модели) и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Мы протестируем данную гипотезу на практическом занятии 2, когда познакомимся с регрессионными моделями с переменными взаимодействия. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Приведен к непрерывной шкале от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения: World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from scipy import stats
import seaborn as sns
import numpy as np
import matplotlib.pyplot as mpl

Откроем массив данных для репликации результатов исследования: lab1.dta

In [ ]:
lab1 = pd.read_stata('lab1.dta')

Познакомимся с тем, как устроен массив данных.

In [ ]:
lab1.head(10)

Ниже представим описательные статистики.

In [ ]:
lab1.describe()

С одной стороны, можно, конечно, по отдельности строить гистограмму для каждой интересующей переменной, как ниже

In [ ]:
mpl.hist(lab1["cpi"], bins = 20, color = "red", edgecolor = "white")

Или диаграмму рассеяния между переменными. К примеру, между зависимой переменной (cpi) и ключевым предиктором - уровнем демократии (dem). Что можно сказать про характер связи между коррупцией и демократией?

In [ ]:
sns.scatterplot(data = lab1, x = "dem", y = "cpi")

Однако это не очень продуктивно. Поэтому лучше сразу построим общий pairplot, демонстрирующий диаграммы рассеяния между переменными попарно (в том числе, учитывая контрольные переменные) и распределения отдельных переменных. Что можно заметить по pairplot? Какие Ваши наблюдения?

In [ ]:
sns.pairplot(lab1)

Дополнительно представим ниже корреляционную матрицу.

In [ ]:
lab1.corr().round(3)

Оценим первую регрессионную модель (m1). Это будет парная линейная регрессия: пока учитываем только ключевой предиктор - уровень демократии, контрольные переменные, предложенные авторами статьи, включим позже.

Проинтерпретируйте полученные результаты.

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem", data = lab1).fit()
print(m1.summary())

Добавим в модель контрольные переменные.

* Критически отнеситесь к выбору контролей авторами. Какие слабые стороны этого выбора можно отметить?
* Сравните результаты m2, полученные по взаимосвязи демократии и коррупции, с соответствующими результатами в m1. Прокомментируйте различия

In [ ]:
m2 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit()
print(m2.summary())

Отдельно можно вывести вектор оценок параметров модели. Если хотите вывести оценку определенного параметра, то в квадратных скобках укажите его номер, не забываем, что в Python отсчет идет с 0, поэтому константе соответствует 0, а не 1.

In [ ]:
m2.params

Также можно вывести ковариационную матрицу оценок параметров. Какая информация в ней содержится? По главной диагонали - дисперсии оценок параметров, остальные элементы - ковариации между оценками коэффициентов (совместная изменчивость). Если взять квадратный корень элементов по главной диагонали, то получим стандартные ошибки оценок коэффициентов, которые нам пригодятся при проверке гипотезы о незначимости коэффициентов в регрессионной модели.    

In [ ]:
m2.cov_params()

Представим полученный объект как матрицу в Python, чтобы можно было обратиться к определенным элементам. И давайте посмотрим, как рассчитать p-value в случае проверки гипотезы о незначимости коэффициента при предикторе dem против двусторонней альтернативы.

Статистика критерия рассчитывается следующим образом: $\dfrac{\hat{b}}{se}$. При верной нулевой гипотезе статистика имеет распределение Стьюдента с количеством степеней свободы равным $n-k-1$, где $k$ - количество предикторов в модели, не забываем про еще один параметр - это константа.

Сделайте вывод на основе полученного p-value.

In [ ]:
Cov_m2 = np.asmatrix(m2.cov_params())
(1-stats.t.cdf(abs(m2.params[1]/Cov_m2[1,1]**0.5), m2.nobs - m2.df_model - 1))*2

Для того же коэффициента построим 95%-ый доверительный интервал. В качестве отправной точки используем оценку соответствующего коэффициента $\hat{\beta}$ при dem. Также для того, чтобы задать границы, нам понадобится стандартная ошибка оценки коэффициента и критическая точка (квантиль по распределению Стьюдента уровня 0.975, $df = n - k - 1$).

Проинтерпретируйте полученный доверительный интервал.

In [ ]:
left_boundary = m2.params[1] - Cov_m2[1,1]**0.5*stats.t.ppf(0.975, m2.nobs - m2.df_model - 1)
right_boundary = m2.params[1] + Cov_m2[1,1]**0.5*stats.t.ppf(0.975, m2.nobs - m2.df_model - 1)

print(left_boundary.round(4), right_boundary.round(4))

Проверим модель на мультиколлинеарность. Для этого рассчитаем VIF - variance inflation factor. Название этого показателя говорит само за себя: это показатель того, во сколько раз "вздувается" дисперсия оценок коэффициентов в условиях мультиколлинерности. Значения VIF 10 и выше принято считать высокими.

Для того, чтобы при построении вспомогательных регрессий учитывалась и константа, добавим в матрицу X столбец из единиц, используя add_constant(X).   

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = lab1.dropna()[["dem", "fp", "stab", "loggdppc", "britcol"]]
X = add_constant(X)

vif_data = pd.DataFrame()
vif_data["parameter"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data)

Проверим, можно ли говорить про гомоскедастичность (здравый смысл подсказывает, что надеяться не стоит), но тем не менее, реализуем определенные диагностики. Для начала построим график, показывающий, как связаны остатки в квадрате и предсказанные значения по модели ($\hat{y}$)

In [ ]:
fitted = m2.predict()
residuals_sq = m2.resid**2
fig, ax = mpl.subplots()
sns.scatterplot(x = fitted, y = residuals_sq)
ax.set_xlabel( "Fitted values")
ax.set_ylabel( "Squared residuals")

Есть и формальные тесты для проверки гипотезы о гомоскедастичности. К примеру, тест Бреуша-Пагана. Давайте его реализуем и проинтепретируем результаты на основе p-value.

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan
bp_test = het_breuschpagan(m2.resid, X)
print(bp_test[1])

Переоценим модель с робастными стандартными ошибками (HC3).

In [ ]:
m3 = smf.ols(formula = "cpi ~ dem + fp + stab + loggdppc + britcol", data = lab1).fit(cov_type = "HC3")
print(m3.summary())

В следующий раз мы продолжим работать с этими данными, а именно, рассмотрим модель с переменной взаимодействия для проверки гипотезы авторов о совместном эффекте свободы прессы и демократии на уровень коррупции.